In [1]:

""" Parser Module for Transfermarkt Crawler."""
import re
from headers import TRANSFERS, PLAYERS, TEAMS, MANAGERS, MANAGER_HISTORY
import collections
def file_read(file_name):
    """ Read files function. """
    try:
        with open(file_name) as file_data:
            return file_data.read()
    except UnicodeDecodeError:
        with open(file_name, encoding='utf-8') as file_data:
            return file_data.read()

In [2]:
def cut_page(start_token, end_token, page):
    """ Cut the page.

        Cut the page in the start_token, and then
        the first token that matchs with the position
        bigger than the position of the start token.

        return cut of the page
    """
    start_pos = [(a.end()) for a in list(re.finditer(start_token, page))]

    if start_pos:
        start_pos = start_pos[0]
        end_pos = [(a.start()) for a in list(re.finditer(end_token, page))]
        end_pos = list(filter(lambda x: x > start_pos, end_pos))[0]

        return page[start_pos:end_pos]

    return page


In [3]:
def _match_positions(start_list, end_list):
    """ Match start and end positions. """

    if len(start_list) == 1:
        value = start_list[0]
        return {value: list(filter(lambda x: value < x, end_list))[0]}

    result = {}
    for start in start_list:
        for end in end_list:
            if start < end:
                result[start] = end
                break

    return result

In [4]:
def file_write(team_info, players_info, managers_info, header):
    """ Write a file with team info.

        players_info = list = each element is a
        dict with players info
        season = int = collect season

        Responsible for create/alterate a file.

        Two files will be change along time:
            transfer.txt - Will store all gathered transfers.
            player.txt - Will store all gathered players
            teams.txt - Will store all gathered teams.
            managers.txt
    """

    with open('Output/teams.txt', 'a+',encoding='utf-8') as file:
        save_file(file, TEAMS, team_info, header)

    with open('Output/players_id.txt', 'a+',encoding='utf-8') as file:
        players_info = verify_id(file, players_info)

    header_aux = header
    with open('Output/players.txt', 'a+',encoding='utf-8') as file:
        for player in players_info:
            save_file(file, PLAYERS, player, header)
            header = False
        
        
        header = header_aux

    header_aux = header
    

    with open('Output/transfers.txt', 'a+',encoding='utf-8') as file:
        transfers = []
        transfers = list(map(lambda x: x['Transfers'],players_info))
        
        #isinstance(transfers, collections.Iterable)
        isinstance(player_transfers, collections.Iterable)
        print(isinstance(player_transfers, collections.Iterable))
        for player_transfers in transfers:
            
            #isinstance(player_transfers, collections.Iterable)
            for transfer in player_transfers:
                
                #isinstance(player_transfers, collections.Iterable)
                save_file(file, TRANSFERS, transfer, header)
                header = False
                
        header = header_aux

    with open('Output/managers_id.txt', 'a+',encoding='utf-8') as file:
        managers_info = verify_id(file, managers_info)

    with open('Output/managers.txt', 'a+',encoding='utf-8') as file:
        for manager in managers_info:
            save_file(file, MANAGERS, manager, header)
            header = False
        header = header_aux

    manager_history = []
    with open('Output/managers_history.txt', 'a+',encoding='utf-8') as file:
        manager_history = list(map(lambda x: x['History'],managers_info))
        for history in manager_history:
            for works in history:
                save_file(file, MANAGER_HISTORY, works, header)
                header = False


In [5]:
print(player_transfers)

NameError: name 'player_transfers' is not defined

In [6]:
import analyzer
import leagues
import players
import managers
import teams

if __name__ == '__main__':

    SEASON_START = 2017
    SEASON_END = 2018
    HEADER = True

    for league in analyzer.file_read("Input/leagues.txt").split('\n'):

        league_teams = leagues.get_teams(league)

        for team in league_teams:

            seasons = []  # new list of seasons of a team

            for season in range(SEASON_START, SEASON_END):

                print("Temporada: ", season)

                team_players = teams.get_players(league_teams[team],
                                                 team, season)

                team_info = teams.get_team_info(league_teams[team],
                                                team, season)
                managers_info = []
                if team_info['Manager'] is not None:
                    for index, manager in enumerate(team_info['Manager']):
                        manager_id = team_info['Manager Id'][index]
                        info = managers.get_manager_info(manager, manager_id)
                        managers_info.append(info)

                players_info = []
                for player in team_players:

                    players_info.append(players.get_player_info(
                                                team_players[player], player))

                analyzer.file_write(team_info, players_info,
                                  managers_info, HEADER)
                HEADER = False


UnsupportedOperation: detach